In [26]:
import scipy.io
import numpy
import torchtext
import torch
from torch.autograd import Variable
import torch.nn as nn
from torchtext.vocab import Vectors, GloVe
import torch.nn.functional as F
import pdb
from copy import deepcopy
import pdb

In [27]:
num_constants = 7
#Background
zero_extension = torch.zeros(1,num_constants)
zero_extension[0,0] = 1
succ_extension = torch.eye(num_constants-1,num_constants-1)
succ_extension = torch.cat((torch.zeros(num_constants-1,1),succ_extension),1)
succ_extension = torch.cat((succ_extension,torch.zeros(1,num_constants)),0)

#Intensional Predicates
aux_extension = torch.zeros(num_constants,num_constants)
even_extension = torch.zeros(1,num_constants)

valuation_init = [Variable(zero_extension), Variable(succ_extension), Variable(aux_extension), Variable(even_extension)]

#Target
target = torch.zeros(num_constants,num_constants)
target[0] = torch.zeros(1,num_constants)
odd = [0,2,4,6]
for integer in odd:
    target[0,integer]=1
 


In [28]:
print(valuation_init[1], target, valuation_init[3])

(Variable containing:
    0     1     0     0     0     0     0
    0     0     1     0     0     0     0
    0     0     0     1     0     0     0
    0     0     0     0     1     0     0
    0     0     0     0     0     1     0
    0     0     0     0     0     0     1
    0     0     0     0     0     0     0
[torch.FloatTensor of size 7x7]
, 
    1     0     1     0     1     0     1
    0     0     0     0     0     0     0
    0     0     0     0     0     0     0
    0     0     0     0     0     0     0
    0     0     0     0     0     0     0
    0     0     0     0     0     0     0
    0     0     0     0     0     0     0
[torch.FloatTensor of size 7x7]
, Variable containing:
    0     0     0     0     0     0     0
[torch.FloatTensor of size 1x7]
)


In [30]:
def decoder_efficient(valuation, step):
    ## Create valuation_new
    valuation_new = [deepcopy(valuation[0]),deepcopy(valuation[1]),Variable(torch.zeros(valuation[2].size())),Variable(torch.zeros(valuation[3].size()))]
                    
    ##Unifications
    rules_aux = torch.cat((rules[:,:num_feat],rules[:,num_feat:2*num_feat],rules[:,2*num_feat:3*num_feat]),0)
    rules_aux = rules_aux.repeat(num_predicates,1)
    embeddings_aux = embeddings.repeat(1,num_rules*3).view(-1,num_feat)    
    #unifs = F.cosine_similarity(embeddings_aux, rules_aux).view(num_predicates,-1)
    
    unifs = F.pairwise_distance(embeddings_aux, rules_aux).view(num_predicates,-1)
    unifs = torch.exp(-unifs)
    unifs_sum = torch.sum(unifs, 0)
    unifs= unifs/unifs_sum

    ##Get_Valuations
    for predicate in intensional_predicates:
        if valuation[predicate].size()[0] == 1:
            for s in range(num_constants):
                valuation_aux = Variable(torch.Tensor([0]))
                for body1 in range(num_predicates):
                    for body2 in range(num_predicates):
                        ## Get nums
                        if valuation[body1].size()[0] == 1:
                            if valuation[body2].size()[0] == 1:
                                num = torch.min(valuation[body1][0,s],valuation[body2][0,s])
                            else: 
                                num = torch.min(valuation[body1][0,:],valuation[body2][:,s])
                                num = torch.max(num)
                        else:
                            if valuation[body2].size()[0] == 1:
                                num = torch.min(valuation[body1][:,s],valuation[body2][0,s])
                                num = torch.max(num)
                            else:
                                num = 0


                        ## max across three rules
                        new = Variable(torch.Tensor([0]))
                        for rule in range(num_rules): 
                            unif = unifs[predicate][rule]*unifs[body1][num_rules+rule]*unifs[body2][2*num_rules+rule]
                            new = torch.max(new,unif) 

                        num = num*new 
                        valuation_aux = torch.max(valuation_aux, num)
                valuation_new[predicate][0,s] = torch.max(valuation[predicate][0,s], valuation_aux) 
            
            
            
        else:
            for s in range(num_constants):
                for o in range(num_constants):
                    valuation_aux = Variable(torch.Tensor([0]))
                    for body1 in range(num_predicates):
                        for body2 in range(num_predicates):
                            ## Get nums
                            if valuation[body1].size()[0] == 1:
                                if valuation[body2].size()[0] == 1:
                                    num = torch.min(valuation[body1][0,s],valuation[body2][0,o])
                                else: 
                                    num = torch.min(valuation[body1][0,s],valuation[body2][s,o])
                                    #num = torch.max(num)
                            else:
                                if valuation[body2].size()[0] == 1:
                                    num = torch.min(valuation[body1][s,o],valuation[body2][0,o])
                                    #num = torch.max(num)
                                else: 
                                    num = torch.min(valuation[body1][s,:],valuation[body2][:,o])
                                    num = torch.max(num)

                            ## max across three rules
                            new = Variable(torch.Tensor([0]))
                            for rule in range(num_rules): 
                                unif = unifs[predicate][rule]*unifs[body1][num_rules+rule]*unifs[body2][2*num_rules+rule]
                                new = torch.max(new,unif)
                                #could be amalgamate

                            num = num*new 
                            valuation_aux = torch.max(valuation_aux, num)
                    valuation_new[predicate][s,o] = torch.max(valuation[predicate][s,o], valuation_aux) 
                
    return valuation_new
            
def amalgamate(x,y):
    return x + y - x*y

def rbf(x,y):
    ans = (x-y)**2
    ans = torch.mean(ans,1)
    ans = torch.exp(-ans)
    return ans



In [36]:
num_iters = 50
learning_rate = .1
steps = 4

num_feat=4
num_rules = 3
num_predicates= 4
intensional_predicates=[2,3]
num_intensional_predicates = len(intensional_predicates)

embeddings = Variable(torch.rand(num_predicates, num_feat), requires_grad=True)
#embeddings = Variable(torch.eye(4), requires_grad=True)

rules = Variable(torch.rand(num_rules, num_feat*3), requires_grad=True)
#rule1 = torch.Tensor([0,0,0,1,1,0,0,0,0,1,0,0]).view(1,-1)
#rule2 = torch.Tensor([0,0,0,1,0,0,0,1,0,0,1,0]).view(1,-1)
#rule3 = torch.Tensor([0,0,1,0,0,1,0,0,0,1,0,0]).view(1,-1)
#rules = Variable(torch.cat((rule1,rule2,rule3),0), requires_grad=True)

optimizer = torch.optim.Adam([rules, embeddings],lr=learning_rate)

criterion = torch.nn.BCELoss(size_average=False)


for i in range(num_iters):
    optimizer.zero_grad()
    
    valuation = valuation_init
    for step in range(steps):
        valuation = decoder_efficient(valuation,step)
        print('step',step,'valuation3', valuation[3], 'valuation2',valuation[2]) 
    loss = criterion(valuation[3][0,:],Variable(torch.Tensor(target[0,:])))
    
    print(i,'lossssssssssssssssssssssssssss',loss.data[0])
    loss.backward()
    optimizer.step()
    
    

('step', 0, 'valuation3', Variable containing:
1.00000e-02 *
  3.2304  2.8671  0.0000  0.0000  0.0000  0.0000  0.0000
[torch.FloatTensor of size 1x7]
, 'valuation2', Variable containing:
1.00000e-02 *
  2.0043  1.2969  1.6575  0.0000  0.0000  0.0000  0.0000
  0.0000  0.0000  0.0000  1.6575  0.0000  0.0000  0.0000
  0.0000  0.0000  0.0000  0.0000  1.6575  0.0000  0.0000
  0.0000  0.0000  0.0000  0.0000  0.0000  1.6575  0.0000
  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  1.6575
  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
[torch.FloatTensor of size 7x7]
)
('step', 1, 'valuation3', Variable containing:
1.00000e-02 *
  3.2304  2.8671  0.1091  0.0608  0.0000  0.0000  0.0000
[torch.FloatTensor of size 1x7]
, 'valuation2', Variable containing:
1.00000e-02 *
  2.0043  1.2969  1.6575  0.0328  0.0228  0.0000  0.0000
  0.0465  0.0411  0.0351  1.6575  0.0328  0.0228  0.0000
  0.0000  0.0000  0.0000  0.0000  1.6575  0.0328  0

KeyboardInterrupt: 

In [8]:
print(embeddings)
print(rules)

Variable containing:
-2.9000 -2.2352  3.9613  3.8525
 2.9210 -0.3001 -2.6078  2.9515
-0.6264 -1.1778 -0.2562 -2.6232
 3.0092  3.6773  2.7466 -0.1558
[torch.FloatTensor of size 4x4]

Variable containing:

Columns 0 to 9 
-0.6106 -1.1431 -0.1900 -2.1985 -0.6681 -1.1533 -0.1946 -2.2965 -0.6220 -1.1119
 0.8267  1.9358  2.2675 -3.5815  4.5003  2.9747 -0.1622  2.7975  1.1742 -2.2246
 2.9964  3.0389  2.8995 -0.1885 -2.1753 -1.6380  3.2507  3.1582 -2.1057 -2.0332

Columns 10 to 11 
-0.1955 -2.2220
-2.9068 -0.2882
 3.4622  2.9245
[torch.FloatTensor of size 3x12]



In [34]:

rules_aux = torch.cat((rules[:,:num_feat],rules[:,num_feat:2*num_feat],rules[:,2*num_feat:3*num_feat]),0)
rules_aux = rules_aux.repeat(num_predicates,1)
embeddings_aux = embeddings.repeat(1,num_rules*3).view(-1,num_feat)


unifs = F.cosine_similarity(embeddings_aux, rules_aux).view(num_predicates,-1)
print('aaaaa',unifs)
print('oooo',F.pairwise_distance(embeddings_aux, rules_aux).view(num_predicates,-1))

unifs = F.pairwise_distance(embeddings_aux, rules_aux).view(num_predicates,-1)
unifs = torch.exp(-unifs)
unifs_sum = torch.sum(unifs, 0)
unifs= unifs/unifs_sum
print('finaaaal',unifs)


('aaaaa', Variable containing:
 0.1133 -0.2670  0.1734  0.7993 -0.5618 -0.4648  0.5997  0.7198 -0.4541
-0.2873  0.9701 -0.0599 -0.4675  0.7327  0.9986 -0.3193 -0.0246  0.9991
-0.8764 -0.0773  0.9992 -0.4089  0.4783 -0.1002  0.9046 -0.3968 -0.0712
 1.0000 -0.2139 -0.8920  0.6829 -0.8128 -0.2494 -0.6567  0.4617 -0.2772
[torch.FloatTensor of size 4x9]
)
('oooo', Variable containing:
 4.6118  4.8784  5.0575  2.3986  4.5802  7.2084  3.5519  2.8634  7.0813
 6.4376  2.7534  6.5002  7.3955  4.0485  0.3396  7.2838  5.1443  0.3994
 6.8246  4.6845  0.1811  6.4932  3.5536  6.3726  1.7700  4.8479  6.2003
 0.0240  4.1483  6.9407  2.7985  4.1024  6.1423  6.5545  2.7343  6.1053
[torch.FloatTensor of size 4x9]
)
('finaaaal', Variable containing:
 0.0100  0.0790  0.0075  0.5904  0.1407  0.0010  0.1426  0.4206  0.0012
 0.0016  0.6612  0.0018  0.0040  0.2395  0.9936  0.0034  0.0430  0.9925
 0.0011  0.0959  0.9895  0.0098  0.3929  0.0024  0.8469  0.0578  0.0030
 0.9872  0.1639  0.0011  0.3958  0.2269  0.00

In [ ]:

input =torch.randn(5,7)
print(input)
print(F.dropout(input, p=.2,training=True))